# Overview

This is the 3rd algorithm on my list. I am going to build this model on previous dataset focusing on `cow` as usual.

In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.pipeline import make_pipeline
from category_encoders import OrdinalEncoder


# Data preparation
### Import